In [ ]:
import cv2
import numpy as np
from tensorflow import keras
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk

# ==================================================
#                 الإعدادات العامة
# ==================================================
MODEL_PATH = "affectnet_final.keras"
CLASS_NAMES_PATH = "class_names.txt"
IMG_SIZE = 96

# ==================================================
#         تحميل نموذج التعلم العميق
# ==================================================
model = keras.models.load_model(MODEL_PATH)

with open(CLASS_NAMES_PATH, "r", encoding="utf-8") as f:
    class_names = [line.strip() for line in f if line.strip()]

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

cap = cv2.VideoCapture(0)

# ==================================================
#              إعداد واجهة Tkinter
# ==================================================
root = tk.Tk()
root.title("AI Emotion Detection System")
root.geometry("1000x650")
root.configure(bg="#1e1e1e")

# ===== عنوان =====
title = tk.Label(root, text="Real-Time Emotion Detection",
                 font=("Arial", 20, "bold"),
                 fg="#00d4ff", bg="#1e1e1e")
title.pack(pady=10)

# ===== إطار الفيديو =====
video_frame = tk.Frame(root, bg="#1e1e1e")
video_frame.pack()

video_label = tk.Label(video_frame)
video_label.pack()

# ==================================================
#              لوحة التحكم
# ==================================================
control_frame = tk.Frame(root, bg="#2b2b2b")
control_frame.pack(fill="x", pady=10)

brightness_var = tk.IntVar(value=50)
blur_var = tk.IntVar(value=1)
threshold_var = tk.IntVar(value=0)
emotion_var = tk.BooleanVar(value=True)

def styled_scale(label_text, variable, from_, to_):
    frame = tk.Frame(control_frame, bg="#2b2b2b")
    frame.pack(side="left", padx=20)
    tk.Label(frame, text=label_text, fg="white",
             bg="#2b2b2b").pack()
    scale = ttk.Scale(frame, from_=from_, to=to_,
                      variable=variable, orient="horizontal", length=150)
    scale.pack()

styled_scale("Brightness", brightness_var, 0, 100)
styled_scale("Blur", blur_var, 0, 20)
styled_scale("Threshold", threshold_var, 0, 255)

# ==================================================
#                 وظائف الأزرار
# ==================================================
def capture_image():
    ret, frame = cap.read()
    if ret:
        cv2.imwrite("captured_image.png", frame)
        print("تم حفظ الصورة ✅")

def reset_values():
    brightness_var.set(50)
    blur_var.set(1)
    threshold_var.set(0)

def toggle_emotion():
    emotion_var.set(not emotion_var.get())

def exit_app():
    cap.release()
    root.destroy()

button_frame = tk.Frame(root, bg="#1e1e1e")
button_frame.pack(pady=10)

tk.Button(button_frame, text="📸 Capture",
          command=capture_image,
          bg="#4CAF50", fg="white",
          width=12).pack(side="left", padx=10)

tk.Button(button_frame, text="🔄 Reset",
          command=reset_values,
          bg="#ff9800", fg="white",
          width=12).pack(side="left", padx=10)

tk.Button(button_frame, text="🧠 Emotion ON/OFF",
          command=toggle_emotion,
          bg="#2196F3", fg="white",
          width=15).pack(side="left", padx=10)

tk.Button(button_frame, text="❌ Exit",
          command=exit_app,
          bg="#f44336", fg="white",
          width=12).pack(side="left", padx=10)

# ==================================================
#              معالجة الفيديو
# ==================================================
def update_frame():
    ret, frame = cap.read()
    if not ret:
        return

    processed = frame.copy()

    # ===== تحسين إضاءة تلقائي (CLAHE) =====
    lab = cv2.cvtColor(processed, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    l = clahe.apply(l)
    lab = cv2.merge((l,a,b))
    processed = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    # ===== إضاءة يدوية =====
    beta = brightness_var.get() - 50
    processed = cv2.convertScaleAbs(processed, alpha=1.0, beta=beta)

    # ===== Blur =====
    blur_value = int(blur_var.get())
    if blur_value > 0:
        if blur_value % 2 == 0:
            blur_value += 1
        processed = cv2.GaussianBlur(processed, (blur_value, blur_value), 0)

    # ===== Threshold =====
    if threshold_var.get() > 0:
        gray_temp = cv2.cvtColor(processed, cv2.COLOR_BGR2GRAY)
        _, thresh_img = cv2.threshold(
            gray_temp, threshold_var.get(), 255, cv2.THRESH_BINARY)
        processed = cv2.cvtColor(thresh_img, cv2.COLOR_GRAY2BGR)

    # ===== كشف الوجه =====
    gray = cv2.cvtColor(processed, cv2.COLOR_BGR2GRAY)

    if emotion_var.get():
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x, y, w, h) in faces:
            roi = processed[y:y+h, x:x+w]
            cv2.rectangle(processed, (x, y),
                          (x+w, y+h), (0,255,0), 2)

            face_resized = cv2.resize(roi, (IMG_SIZE, IMG_SIZE))
            face_array = face_resized.astype("float32") / 255.0
            face_array = np.expand_dims(face_array, axis=0)

            prediction = model.predict(face_array, verbose=0)[0]
            best_index = np.argmax(prediction)
            emotion = class_names[best_index]
            confidence = prediction[best_index] * 100

            cv2.putText(processed,
                        f"{emotion} ({confidence:.1f}%)",
                        (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.7, (0,255,0), 2)

    # ===== عرض داخل Tk =====
    img_rgb = cv2.cvtColor(processed, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img_rgb)
    imgtk = ImageTk.PhotoImage(image=img)

    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)

    root.after(10, update_frame)

update_frame()
root.mainloop()
